<img src="http://developer.download.nvidia.com/notebooks/dlsw-notebooks/tensorrt_torchtrt_efficientnet/nvidia_logo.png" width="90px">

# PySpark Huggingface Inferencing
### Sentiment Analysis using Pipelines with PyTorch

In this notebook, we demonstrate distributed inference with Huggingface Pipelines to perform sentiment analysis.  
From: https://huggingface.co/docs/transformers/quicktour#pipeline-usage

In [1]:
import torch
from transformers import pipeline

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
classifier = pipeline("sentiment-analysis", device=device)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda


In [4]:
classifier(("We are very happy to show you the 🤗 Transformers library."))

[{'label': 'POSITIVE', 'score': 0.9997795224189758}]

In [5]:
results = classifier(["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309


Let's try a different model and tokenizer in the pipeline.

In [6]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=device)
classifier("Nous sommes très heureux de vous présenter la bibliothèque 🤗 Transformers.")

Device set to use cuda


[{'label': '5 stars', 'score': 0.7272652983665466}]

## PySpark

In [9]:
from pyspark.sql.functions import col, struct, pandas_udf
from pyspark.ml.functions import predict_batch_udf
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark import SparkConf

In [10]:
import os
import json
import pandas as pd
import datasets
from datasets import load_dataset
datasets.disable_progress_bars()

Check the cluster environment to handle any platform-specific Spark configurations.

In [11]:
on_databricks = os.environ.get("DATABRICKS_RUNTIME_VERSION", False)
on_dataproc = os.environ.get("DATAPROC_IMAGE_VERSION", False)
on_standalone = not (on_databricks or on_dataproc)

#### Create Spark Session

For local standalone clusters, we'll connect to the cluster and create the Spark Session.  
For CSP environments, Spark will either be preconfigured (Databricks) or we'll need to create the Spark Session (Dataproc).

In [12]:
conf = SparkConf()

if 'spark' not in globals():
    if on_standalone:
        import socket
        conda_env = os.environ.get("CONDA_PREFIX")
        hostname = socket.gethostname()
        conf.setMaster(f"spark://{hostname}:7077")
        conf.set("spark.pyspark.python", f"{conda_env}/bin/python")
        conf.set("spark.pyspark.driver.python", f"{conda_env}/bin/python")

    conf.set("spark.executor.cores", "8")
    conf.set("spark.task.resource.gpu.amount", "0.125")
    conf.set("spark.executor.resource.gpu.amount", "1")
    conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
    conf.set("spark.python.worker.reuse", "true")

conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1000")
spark = SparkSession.builder.appName("spark-dl-examples").config(conf=conf).getOrCreate()
sc = spark.sparkContext

25/02/04 13:23:47 WARN Utils: Your hostname, cb4ae00-lcedt resolves to a loopback address: 127.0.1.1; using 10.110.47.100 instead (on interface eno1)
25/02/04 13:23:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/04 13:23:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
dataset = load_dataset("imdb", split="test")
dataset = dataset.to_pandas().drop(columns="label")

#### Create PySpark DataFrame

In [14]:
df = spark.createDataFrame(dataset).repartition(8)
df.schema

StructType([StructField('text', StringType(), True)])

In [15]:
df.count()

25000

In [16]:
df.take(1)

25/02/04 13:23:54 WARN TaskSetManager: Stage 6 contains a task of very large size (4021 KiB). The maximum recommended task size is 1000 KiB.


[Row(text="Anyone remember the first CKY, CKY2K etc..? Back when it was about making crazy cool stuff, rather than watching Bam Margera act like a douchebag, spoiled 5 year old, super/rock-star wannabe.<br /><br />The show used to be awesome, however, Bam's fame and wealth has led him to believe, that we now enjoy him acting childish and idiotic, more than actual cool stuff, that used to be in ex. CKY2K.<br /><br />The acts are so repetitive, there's like nothing new, except annoying stupidity and rehearsed comments... The only things we see is Bam Margera, so busy showing us how much he doesn't care, how much money he got or whatsoever.<br /><br />I really got nothing much left to say except, give us back CKY2K, cause Bam suck..<br /><br />I enjoy watching Steve-o, Knoxville etc. a thousand times more.")]

In [17]:
data_path = "spark-dl-datasets/imdb_test"
if on_databricks:
    dbutils.fs.mkdirs("/FileStore/spark-dl-datasets")
    data_path = "dbfs:/FileStore/" + data_path

df.write.mode("overwrite").parquet(data_path)

25/02/04 13:23:54 WARN TaskSetManager: Stage 9 contains a task of very large size (4021 KiB). The maximum recommended task size is 1000 KiB.


#### Load and preprocess DataFrame

Define our preprocess function. We'll take the first sentence from each sample as our input for sentiment analysis.

In [18]:
@pandas_udf("string")
def preprocess(text: pd.Series) -> pd.Series:
    return pd.Series([s.split(".")[0] for s in text])

In [19]:
# Limit to N rows, since this can be slow
df = spark.read.parquet(data_path).limit(256).repartition(8)
df = df.select(preprocess(col("text")).alias("input")).cache()
df.show(truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                               input|
+----------------------------------------------------------------------------------------------------+
|Doesn't anyone bother to check where this kind of sludge comes from before blathering on about it...|
|                          There were two things I hated about WASTED : The directing and the script |
|                                I'm rather surprised that anybody found this film touching or moving|
|Cultural Vandalism Is the new Hallmark production of Gulliver's Travels an act of cultural vandal...|
|I was at Wrestlemania VI in Toronto as a 10 year old, and the event I saw then was pretty differe...|
|                                                                     This movie has been done before|
|[ as a new resolution for this year 2005, i decide to write a comment fo

## Inference using Spark DL API

Distributed inference using the PySpark [predict_batch_udf](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.functions.predict_batch_udf.html#pyspark.ml.functions.predict_batch_udf):

- predict_batch_fn uses PyTorch APIs to load the model and return a predict function which operates on numpy arrays 
- predict_batch_udf will convert the Spark DataFrame columns into numpy input batches for the predict function

In [20]:
def predict_batch_fn():
    import torch
    from transformers import pipeline
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    pipe = pipeline("sentiment-analysis", device=device)
    def predict(inputs):
        return pipe(inputs.tolist())
    return predict

In [21]:
classify = predict_batch_udf(predict_batch_fn,
                             return_type=StructType([
                                 StructField("label", StringType(), True),
                                 StructField("score", FloatType(), True)
                             ]),
                             batch_size=32)

In [22]:
%%time
# first pass caches model/fn
# note: expanding the "struct" return_type to top-level columns
preds = df.withColumn("preds", classify(struct("input"))).select("input", "preds.*")
results = preds.collect()

CPU times: user 8.82 ms, sys: 2.5 ms, total: 11.3 ms
Wall time: 3.59 s


In [23]:
%%time
preds = df.withColumn("preds", classify("input")).select("input", "preds.*")
results = preds.collect()

CPU times: user 3.19 ms, sys: 1.65 ms, total: 4.84 ms
Wall time: 392 ms


In [24]:
%%time
preds = df.withColumn("preds", classify(col("input"))).select("input", "preds.*")
results = preds.collect()

CPU times: user 3.43 ms, sys: 2.33 ms, total: 5.77 ms
Wall time: 403 ms


In [25]:
preds.show(truncate=80)

+--------------------------------------------------------------------------------+--------+----------+
|                                                                           input|   label|     score|
+--------------------------------------------------------------------------------+--------+----------+
|Doesn't anyone bother to check where this kind of sludge comes from before bl...|NEGATIVE| 0.9984042|
|      There were two things I hated about WASTED : The directing and the script |NEGATIVE| 0.9979019|
|            I'm rather surprised that anybody found this film touching or moving|POSITIVE|  0.839279|
|Cultural Vandalism Is the new Hallmark production of Gulliver's Travels an ac...|NEGATIVE|0.99726933|
|I was at Wrestlemania VI in Toronto as a 10 year old, and the event I saw the...|POSITIVE|0.98212504|
|                                                 This movie has been done before|NEGATIVE| 0.9419482|
|[ as a new resolution for this year 2005, i decide to write a comment fo

## Using Triton Inference Server
In this section, we demonstrate integration with the [Triton Inference Server](https://developer.nvidia.com/nvidia-triton-inference-server), an open-source, GPU-accelerated serving solution for DL.  
We use [PyTriton](https://github.com/triton-inference-server/pytriton), a Flask-like framework that handles client/server communication with the Triton server.  

The process looks like this:
- Distribute a PyTriton task across the Spark cluster, instructing each node to launch a Triton server process.
- Define a Triton inference function, which contains a client that binds to the local server on a given node and sends inference requests.
- Wrap the Triton inference function in a predict_batch_udf to launch parallel inference requests using Spark.
- Finally, distribute a shutdown signal to terminate the Triton server processes on each node.

<img src="../images/spark-server.png" alt="drawing" width="700"/>

In [26]:
from functools import partial

Import the helper class from server_utils.py:

In [32]:
sc.addPyFile("server_utils.py")

from server_utils import TritonServerManager

Define the Triton Server function:

In [33]:
def triton_server(ports):
    import time
    import signal
    import numpy as np
    import torch
    from transformers import pipeline
    from pytriton.decorators import batch
    from pytriton.model_config import DynamicBatcher, ModelConfig, Tensor
    from pytriton.triton import Triton, TritonConfig
    from pyspark import TaskContext

    print(f"SERVER: Initializing pipeline on worker {TaskContext.get().partitionId()}.")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    pipe = pipeline("sentiment-analysis", device=device)
    print(f"SERVER: Using {device} device.")

    @batch
    def _infer_fn(**inputs):
        sentences = np.squeeze(inputs["text"]).tolist()
        print(f"SERVER: Received batch of size {len(sentences)}")
        decoded_sentences = [s.decode("utf-8") for s in sentences]
        return {
            "outputs": np.array([[json.dumps(o)] for o in pipe(decoded_sentences)])
        }

    workspace_path = f"/tmp/triton_{time.strftime('%m_%d_%M_%S')}"
    triton_conf = TritonConfig(http_port=ports[0], grpc_port=ports[1], metrics_port=ports[2])
    with Triton(config=triton_conf, workspace=workspace_path) as triton:
        triton.bind(
            model_name="SentimentAnalysis",
            infer_func=_infer_fn,
            inputs=[
                Tensor(name="text", dtype=object, shape=(-1,)),
            ],
            outputs=[
                Tensor(name="outputs", dtype=object, shape=(-1,)),
            ],
            config=ModelConfig(
                max_batch_size=64,
                batcher=DynamicBatcher(max_queue_delay_microseconds=5000),  # 5ms
            ),
            strict=True,
        )

        def _stop_triton(signum, frame):
            # The server manager sends SIGTERM to stop the server; this function ensures graceful cleanup.
            print("SERVER: Received SIGTERM. Stopping Triton server.")
            triton.stop()

        signal.signal(signal.SIGTERM, _stop_triton)

        print("SERVER: Serving inference")
        triton.serve()

#### Start Triton servers

The `TritonServerManager` will handle the lifecycle of Triton server instances across the Spark cluster:
- Find available ports for HTTP/gRPC/metrics
- Deploy a server on each node via stage-level scheduling
- Gracefully shutdown servers across nodes

In [35]:
model_name = "SentimentAnalysis"
server_manager = TritonServerManager(model_name=model_name)

In [ ]:
# Returns {'hostname', (server_pid, [http_port, grpc_port, metrics_port])}
server_manager.start_servers(triton_server)

2025-02-07 11:03:44,809 - INFO - Requesting stage-level resources: (cores=5, gpu=1.0)
2025-02-07 11:03:44,810 - INFO - Starting 1 servers.


{'cb4ae00-lcedt': (2020631, [7000, 7001, 7002])}

#### Define client function

Get the hostname -> url mapping from the server manager:

In [ ]:
host_to_http_url = server_manager.host_to_http_url  # or server_manager.host_to_grpc_url

Define the Triton inference function, which returns a predict function for batch inference through the server:

In [38]:
def triton_fn(model_name, host_to_url):
    import socket
    import numpy as np
    from pytriton.client import ModelClient

    url = host_to_url[socket.gethostname()]
    print(f"Connecting to Triton model {model_name} at {url}.")

    def infer_batch(inputs):
        with ModelClient(url, model_name, inference_timeout_s=240) as client:
            flattened = np.squeeze(inputs).tolist()
            # Encode batch
            encoded_batch = [[text.encode("utf-8")] for text in flattened]
            encoded_batch_np = np.array(encoded_batch, dtype=np.bytes_)
            # Run inference
            result_data = client.infer_batch(encoded_batch_np)
            result_data = np.squeeze(result_data["outputs"], -1)
            return [json.loads(o) for o in result_data]
        
    return infer_batch

In [41]:
classify = predict_batch_udf(partial(triton_fn, model_name=model_name, host_to_url=host_to_http_url),
                             return_type=StructType([
                                 StructField("label", StringType(), True),
                                 StructField("score", FloatType(), True)
                             ]),
                             input_tensor_shapes=[[1]],
                             batch_size=32)

#### Load and preprocess DataFrame

In [39]:
@pandas_udf("string")
def preprocess(text: pd.Series) -> pd.Series:
    return pd.Series([s.split(".")[0] for s in text])

In [40]:
df = spark.read.parquet(data_path).limit(256).repartition(8)
df = df.select(preprocess(col("text")).alias("input")).cache()

25/02/04 13:24:35 WARN CacheManager: Asked to cache already cached data.


#### Run Inference

In [42]:
%%time
# first pass caches model/fn
# note: expanding the "struct" return_type to top-level columns
preds = df.withColumn("preds", classify(struct("input"))).select("input", "preds.*")
results = preds.collect()

CPU times: user 10.5 ms, sys: 2.2 ms, total: 12.7 ms
Wall time: 671 ms


In [43]:
%%time
preds = df.withColumn("preds", classify("input")).select("input", "preds.*")
results = preds.collect()

CPU times: user 1.68 ms, sys: 1.87 ms, total: 3.55 ms
Wall time: 396 ms


In [44]:
%%time
preds = df.withColumn("preds", classify(col("input"))).select("input", "preds.*")
results = preds.collect()

CPU times: user 3.06 ms, sys: 5.02 ms, total: 8.08 ms
Wall time: 408 ms


In [45]:
preds.show(truncate=70)

+----------------------------------------------------------------------+--------+----------+
|                                                                 input|   label|     score|
+----------------------------------------------------------------------+--------+----------+
|Doesn't anyone bother to check where this kind of sludge comes from...|NEGATIVE| 0.9984042|
|There were two things I hated about WASTED : The directing and the ...|NEGATIVE| 0.9979019|
|  I'm rather surprised that anybody found this film touching or moving|POSITIVE|  0.839279|
|Cultural Vandalism Is the new Hallmark production of Gulliver's Tra...|NEGATIVE|0.99726933|
|I was at Wrestlemania VI in Toronto as a 10 year old, and the event...|POSITIVE|0.98212504|
|                                       This movie has been done before|NEGATIVE| 0.9419482|
|[ as a new resolution for this year 2005, i decide to write a comme...|NEGATIVE|0.99678314|
|This movie is over hyped!! I am sad to say that I manage to watch t..

#### Shut down server on each executor

In [46]:
server_manager.stop_servers()

2025-02-04 13:24:40,325 - INFO - Requesting stage-level resources: (cores=5, gpu=1.0)


2025-02-04 13:24:45,576 - INFO - Sucessfully stopped 1 servers.                 


[True]

In [47]:
if not on_databricks: # on databricks, spark.stop() puts the cluster in a bad state
    spark.stop()